In [2]:
import pyspark
import pyspark.sql.functions as F

In [3]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

22/11/06 16:44:42 WARN Utils: Your hostname, MBP-MT.local resolves to a loopback address: 127.0.0.1; using 192.168.0.192 instead (on interface en0)
22/11/06 16:44:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/11/06 16:44:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [12]:
df = spark\
    .read\
    .option("header", "true")\
    .option("sep", ";")\
    .option('encoding', 'latin1')\
    .csv("turno=2")

In [16]:
df.registerTempTable("segundo_turno")

In [13]:
df.printSchema()

root
 |-- DT_GERACAO: string (nullable = true)
 |-- HH_GERACAO: string (nullable = true)
 |-- ANO_ELEICAO: string (nullable = true)
 |-- CD_TIPO_ELEICAO: string (nullable = true)
 |-- NM_TIPO_ELEICAO: string (nullable = true)
 |-- CD_PLEITO: string (nullable = true)
 |-- DT_PLEITO: string (nullable = true)
 |-- NR_TURNO: string (nullable = true)
 |-- CD_ELEICAO: string (nullable = true)
 |-- DS_ELEICAO: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- CD_MUNICIPIO: string (nullable = true)
 |-- NM_MUNICIPIO: string (nullable = true)
 |-- NR_ZONA: string (nullable = true)
 |-- NR_SECAO: string (nullable = true)
 |-- NR_LOCAL_VOTACAO: string (nullable = true)
 |-- CD_CARGO_PERGUNTA: string (nullable = true)
 |-- DS_CARGO_PERGUNTA: string (nullable = true)
 |-- NR_PARTIDO: string (nullable = true)
 |-- SG_PARTIDO: string (nullable = true)
 |-- NM_PARTIDO: string (nullable = true)
 |-- DT_BU_RECEBIDO: string (nullable = true)
 |-- QT_APTOS: string (nullable = true)
 |-- Q

In [75]:
print("numero de secoes:")
df.count()

numero de secoes:


2841231

### Exemplo de query usando spark sql:

In [34]:
spark.sql("""select NM_MUNICIPIO,
                    NR_ZONA, 
                    NR_SECAO, 
                    DS_CARGO_PERGUNTA, 
                    NM_VOTAVEL, 
                    QT_VOTOS, 
                    QT_COMPARECIMENTO, 
                    QT_ABSTENCOES 
             from segundo_turno
""").show()

+------------+-------+--------+-----------------+---------------+--------+-----------------+-------------+
|NM_MUNICIPIO|NR_ZONA|NR_SECAO|DS_CARGO_PERGUNTA|     NM_VOTAVEL|QT_VOTOS|QT_COMPARECIMENTO|QT_ABSTENCOES|
+------------+-------+--------+-----------------+---------------+--------+-----------------+-------------+
|   SÃO PAULO|      1|       1|       Presidente|           Nulo|      12|              279|           91|
|   SÃO PAULO|      1|       1|       Presidente| JAIR BOLSONARO|     132|              279|           91|
|   SÃO PAULO|      1|       1|       Presidente|           LULA|     127|              279|           91|
|   SÃO PAULO|      1|       1|       Presidente|         Branco|       8|              279|           91|
|   SÃO PAULO|      1|       1|       Governador|           Nulo|      10|              279|           91|
|   SÃO PAULO|      1|       1|       Governador|         Branco|      10|              279|           91|
|   SÃO PAULO|      1|       1|      

### Agrupando votos por seções

In [55]:
secoes = spark.sql("""SELECT SG_UF,
                             NM_MUNICIPIO,
                             NR_ZONA, 
                             NR_SECAO, 
                             DS_CARGO_PERGUNTA, 
                             NM_VOTAVEL, 
                             bigint(QT_VOTOS), 
                             bigint(QT_COMPARECIMENTO), 
                             bigint(QT_ABSTENCOES) 
                      FROM segundo_turno
                      WHERE DS_CARGO_PERGUNTA='Presidente'
""")\
.groupBy("SG_UF", "NM_MUNICIPIO","NR_ZONA","NR_SECAO", "QT_COMPARECIMENTO", "QT_ABSTENCOES")\
.pivot("NM_VOTAVEL", ["JAIR BOLSONARO", "LULA", "Branco", "Nulo"])\
.sum("QT_VOTOS")\
.na.fill(0)\
.cache()

secoes.show()

+-----+--------------------+-------+--------+-----------------+-------------+--------------+----+------+----+
|SG_UF|        NM_MUNICIPIO|NR_ZONA|NR_SECAO|QT_COMPARECIMENTO|QT_ABSTENCOES|JAIR BOLSONARO|LULA|Branco|Nulo|
+-----+--------------------+-------+--------+-----------------+-------------+--------------+----+------+----+
|   SP|           SÃO PAULO|      1|     469|              271|           61|            74| 188|     4|   5|
|   SP|           SÃO PAULO|      2|     355|              337|           60|           139| 176|     8|  14|
|   SP|             CONCHAL|     75|      34|              285|           53|           205|  68|     3|   9|
|   SP|SÃO JOSÉ DO RIO P...|    124|      46|              265|          134|           163|  89|     3|  10|
|   SP|           AMERICANA|    158|     133|              232|           95|           170|  54|     2|   6|
|   SP|            MIRACATU|    218|      61|              271|          136|           186|  73|     4|   8|
|   SP|   

In [92]:
# Checagem de sanidade para agrupamento, verificando se não duplicamos votos:
assert secoes.select("SG_UF", "NM_MUNICIPIO", "NR_ZONA", "NR_SECAO").count() == \
       secoes.select("SG_UF", "NM_MUNICIPIO", "NR_ZONA", "NR_SECAO").distinct().count()

### Seções com zero votos de Lula e de Bolsonaro

In [65]:
secoes.filter(F.col("JAIR BOLSONARO") == 0).count()

144

In [74]:
secoes.filter(F.col("JAIR BOLSONARO") == 0)\
.orderBy("SG_UF", "NM_MUNICIPIO", "NR_SECAO")\
.show(200, False)

+-----+-------------------------+-------+--------+-----------------+-------------+--------------+----+------+----+
|SG_UF|NM_MUNICIPIO             |NR_ZONA|NR_SECAO|QT_COMPARECIMENTO|QT_ABSTENCOES|JAIR BOLSONARO|LULA|Branco|Nulo|
+-----+-------------------------+-------+--------+-----------------+-------------+--------------+----+------+----+
|AC   |MARECHAL THAUMATURGO     |4      |326     |160              |90           |0             |153 |1     |6   |
|AC   |MARECHAL THAUMATURGO     |4      |384     |105              |80           |0             |101 |0     |4   |
|AM   |ATALAIA DO NORTE         |42     |25      |149              |45           |0             |138 |0     |11  |
|AM   |ATALAIA DO NORTE         |42     |28      |132              |65           |0             |127 |1     |4   |
|AM   |ATALAIA DO NORTE         |42     |29      |201              |134          |0             |175 |2     |24  |
|AM   |ATALAIA DO NORTE         |42     |37      |108              |17          

In [95]:
secoes.filter(F.col("LULA") == 0).count()

5

In [94]:
secoes.filter(F.col("LULA") == 0)\
.orderBy("SG_UF", "NM_MUNICIPIO", "NR_SECAO")\
.show(200, False)

+-----+--------------+-------+--------+-----------------+-------------+--------------+----+------+----+
|SG_UF|NM_MUNICIPIO  |NR_ZONA|NR_SECAO|QT_COMPARECIMENTO|QT_ABSTENCOES|JAIR BOLSONARO|LULA|Branco|Nulo|
+-----+--------------+-------+--------+-----------------+-------------+--------------+----+------+----+
|PA   |CHAVES        |17     |41      |40               |23           |39            |0   |1     |0   |
|RS   |CHARRUA       |100    |15      |82               |9            |79            |0   |1     |2   |
|ZZ   |CARACAS       |1      |78      |5                |559          |5             |0   |0     |0   |
|ZZ   |CARACAS       |1      |79      |1                |677          |1             |0   |0     |0   |
|ZZ   |CIUDAD GUAYANA|1      |91      |0                |75           |0             |0   |0     |0   |
+-----+--------------+-------+--------+-----------------+-------------+--------------+----+------+----+

